# Agent Supervisor

Inspired by [LangChain](https://github.com/hwchase17/langchain) [Agent Supervisor](https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/agent_supervisor.ipynb), The notebook showcases an AgentSupervisor powered by [Anthropic's Claude](https://www.anthropic.com/news/claude-3-family).


Below, we will create an agent group, with an agent supervisor to help delegate tasks.

![diagram](./img/supervisor-diagram.png)

Before we build, let's configure our environment:

In [1]:
# %%capture --no-stderr
# %pip install -U langchain langchain_anthropic langchain_experimental langgraph pandas

In [2]:
from getpass import getpass
import os


def _set_if_undefined(var: str):
  if not os.environ.get(var):
    os.environ[var] = getpass(f"Please provide your {var}")

_set_if_undefined("ANTHROPIC_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

## Create tools

For this example, you will make an agent to do web research with a search engine. Define the tools they'll use below:

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=5)

tools = [ tavily_tool ]

## Construct Graph

We're ready to start instantiating and invoking the graph using the AgentSupervisor class.

In [4]:
import sys
import logging
from langchain.callbacks.stdout import StdOutCallbackHandler

from slangchain.llms.chat_anthropic_functions import ChatAnthropicFunctions
from slangchain.graphs.anthropic.agent_supervisor import AgentSupervisor

llm = ChatAnthropicFunctions(model_name = "claude-3-haiku-20240307")

stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(stream_handler)

callbacks = [StdOutCallbackHandler()]

agent_super = AgentSupervisor.from_llm_and_tools(
  llm = llm,
  tools = tools,
  early_stopping_method = "force",
  recursion_limit = 50,
  callbacks = callbacks,
  verbosity = True,
)
result = agent_super.invoke(input={"input": "Who is Apple."})
logger.info("RESULT: %s", result)



> Entering new AgentSupervisor chain...
{'supervisor': {'next': 'tavily_search_results_json'}}
----


> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Apple'}`
responded: Let me search for information on Apple:

[{'url': 'https://www.apple.com/', 'content': 'Explore the latest products and services from Apple, including the new iPhone 15 Pro with titanium design and spatial computing. Trade in your old device and get up to 3% Daily Cash back with Apple Card.'}, {'url': 'https://support.apple.com/iphone', 'content': "iPhone Support\nGet started\nUpgrade to iOS\xa017\nIt's easy to upgrade to iOS\xa017 and experience new features to personalize your iPhone, express yourself in calls and messages, share content, and more.\n See how to set up a new eSIM or transfer one from your previous iPhone.\nUse USB-C on iPhone\xa015 models\nYou can charge your iPhone\xa015 and connect to other devices with the USB-C connector.\n Transfer data to a new iP